In [2]:
from ipstartup import *
import geopandas as gpd
from mapboxer import Map
from mapboxer.elections import get
from mapboxer.utils import replace_quotes, iframe, points2border

[cellevents:43:INFO]:starting (time=Aug-20 10:32 MainProcess)


time: 15 ms


In [11]:
# get voter data
voters = pd.read_csv("../data/voters.csv")
border = voters = voters[voters.longitude.notnull() & voters.latitude.notnull()]
voters = voters.rename(columns=dict(localauthorityname="authority", pollingdistrictname="pd", longitude="lon", latitude="lat"))
replace_quotes(voters)
voters = gpd.GeoDataFrame(voters, geometry=gpd.points_from_xy(voters.lon, voters.lat))

[cellevents:43:INFO]:starting (time=Aug-20 10:35 MainProcess)


time: 634 ms


In [3]:
# get borders
pds = points2border(voters, ["authority", "wardname", "pd"])

[cellevents:43:INFO]:starting (time=Aug-19 14:50 MainProcess)


time: 31.6 s


In [4]:
# clip to constituency
constituencies = get.constituencies()
constituencies = constituencies[constituencies.const=="West Dorset"]
border = constituencies.dissolve("const")
pds = gpd.clip(pds, border).reset_index()

[cellevents:43:INFO]:starting (time=Aug-19 14:50 MainProcess)


time: 4.69 s


In [5]:
# derived data and minimise
wards = pds.dissolve(by=["authority", "wardname"])[["geometry"]]
pds_c = pds.copy()[["pd", "geometry"]]
pds_c.geometry = pds_c.centroid
pds = pds[["geometry"]]

[cellevents:43:INFO]:starting (time=Aug-19 14:50 MainProcess)


time: 233 ms


# map

In [8]:
m = Map()
m.center = [-2.58, 50.7977]
m.zoom = 10

m.add_source("wards", wards)
m.add_source("pds", pds)
m.add_source("pds_c", pds_c)

m.add_layer("wards", type="line", source="wards", paint=dict(line_width=3))
m.add_layer("Polling_district", type="line", source="pds")
m.add_layer("PD_names", type="symbol", source="pds_c", property="pd")
m.add_layer_selector()
m.title = "Polling district and ward boundaries generated from a list of geocoded voters"
iframe(m.html())

[cellevents:43:INFO]:starting (time=Aug-19 14:51 MainProcess)


time: 457 ms


In [9]:
m.save("pollingdistricts")

[cellevents:43:INFO]:starting (time=Aug-19 14:53 MainProcess)


time: 17 ms
